<a href="https://colab.research.google.com/github/hatefap/CrackingTheInterview/blob/master/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip  install transformers numpy torch sklearn

     |████████████████████████████████| 2.0MB 25.9MB/s 
     |████████████████████████████████| 890kB 45.8MB/s 
     |████████████████████████████████| 3.2MB 50.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=1d090fc131502151e013fbafe43c0b35037ff1d0edc47de2f45f885b4c285328
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import numpy as np
import random
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd

In [ ]:
def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).

    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf

        tf.random.set_seed(seed)

set_seed(101)

In [ ]:
model_name = "bert-base-uncased"
# max sequence length for each document/sentence sample
max_length = 512

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)


In [ ]:
farsi_path = "drive/MyDrive/final_final.csv"
english_path = "drive/MyDrive/my_final_english_hate.csv"
eng = pd.read_csv(english_path)
farsi = pd.read_csv(farsi_path)
eng.dropna(inplace=True)
farsi.dropna(inplace=True)

In [ ]:
X_train = eng[['tweet']]
Y_train = eng[['class']]

X_google = farsi[['google']]
Y_google = farsi[['ref3']]
Y_google.columns = ['class']

X_bart = farsi[['mbart']]
Y_bart = farsi[['ref3']]
Y_bart.columns = ['class']

In [ ]:
X_google_tune, X_google_test, Y_google_tune, Y_google_test = train_test_split(X_google, Y_google, test_size=0.4, random_state=101)

X_mbart_tune, X_mbart_test  , Y_mbart_tune , Y_mbart_test  = train_test_split(X_bart, Y_bart ,test_size=0.4, random_state=101)

In [ ]:
X_train.head()

,tweet
0,"""@USER @USER @USER queer"" gaywad"
1,"""@USER @USER alsarabsss"" hes a beaner smh you ..."
2,"""@USER @USER you're fucking gay, blacklisted h..."
3,"""@USER LMFAOOOO I HATE BLACK PEOPLE https://t...."
4,"""@USER ""At least I'm not a nigger"" http://t.co..."


In [ ]:
X_google_tune.head()

,google
4840,Mirage night was intercepted leave us\nOur hea...
2343,I recommend not to see horror films because I ...
4699,Jovin date and the number of tweets and follow...
4228,Mr. Frdvsykhvany of Keykā'ūs and Afrasiab he c...
4911,"You are a child of my parents, I realized that..."


In [ ]:
X_mbart_tune.head()

,mbart
4840,"Leaving the night was our poison, the game was..."
2343,"I recommend you don't watch a horror movie, be..."
4699,The history of June and the number of tweets a...
4228,Mr. Iranian reading from Kafka and Arabic make...
4911,"When I was a kid, the first difference I reali..."


In [ ]:
# tokenize the dataset, truncate when passed `max_length`, 
# and pad with 0's when less than `max_length`
train_encodings = tokenizer(X_train['tweet'].to_list(), truncation=True, padding=True, max_length=max_length)
google_valid_encodings = tokenizer(X_google_tune['google'].to_list(), truncation=True, padding=True, max_length=max_length)
mbart_valid_encodings = tokenizer(X_mbart_tune['mbart'].to_list(), truncation=True, padding=True, max_length=max_length)

In [ ]:
class toTorchConverter(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

# convert our tokenized data into a torch Dataset
train_dataset = toTorchConverter(train_encodings, Y_train['class'].to_list())
google_dataset_valid = toTorchConverter(google_valid_encodings, Y_google_tune['class'].astype(dtype="long").to_list())
bart_dataset_valid = toTorchConverter(mbart_valid_encodings, Y_mbart_tune['class'].astype(dtype="long").to_list())

In [ ]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2).to("cuda")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
model2 = BertForSequenceClassification.from_pretrained(model_name, num_labels=2).to("cuda")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

In [ ]:
training_args_b = TrainingArguments(
    output_dir='./results_bart',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs_bart',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=200,               # log & save weights each logging_steps
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)

In [ ]:
training_args_g = TrainingArguments(
    output_dir='./results_google',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs_google',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=200,               # log & save weights each logging_steps
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)

In [ ]:
trainer_google = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args_g,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=google_dataset_valid,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

In [ ]:
trainer_bart = Trainer(
    model=model2,                         # the instantiated Transformers model to be trained
    args=training_args_b,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=bart_dataset_valid,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

In [ ]:
# train the model
trainer_google.train()

Step,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
200,0.600200,0.478021,0.789153,46.993600,66.307000
400,0.432200,0.595690,0.714056,46.678300,66.755000
600,0.414300,0.422082,0.853338,46.699500,66.724000
800,0.358800,0.599565,0.809050,46.737500,66.670000
1000,0.330800,0.656672,0.782092,46.795700,66.587000
1200,0.352900,0.488534,0.855905,46.962800,66.350000
1400,0.305500,0.692879,0.771181,47.071100,66.198000
1600,0.235200,0.581584,0.821566,46.812200,66.564000
1800,0.204700,0.690081,0.811297,46.651700,66.793000
2000,0.191900,0.706315,0.816752,46.622800,66.834000


TrainOutput(global_step=2037, training_loss=0.3404766381459899, metrics={'train_runtime': 2606.9442, 'train_samples_per_second': 0.781, 'total_flos': 7122235291148196.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 161338039, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 1384340, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 1660100, 'train_mem_gpu_alloc_delta': 1781358080, 'train_mem_cpu_peaked_delta': 94789511, 'train_mem_gpu_peaked_delta': 6275048448})

In [ ]:
trainer_google.evaluate()


{'epoch': 3.0,
 'eval_accuracy': 0.8533376123234917,
 'eval_loss': 0.4220818877220154,
 'eval_mem_cpu_alloc_delta': 124148,
 'eval_mem_cpu_peaked_delta': 224693,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 186651648,
 'eval_runtime': 39.1923,
 'eval_samples_per_second': 79.505}

In [ ]:
model_path = "drive/MyDrive/google-bert-base-uncased"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('drive/MyDrive/google-bert-base-uncased/tokenizer_config.json',
 'drive/MyDrive/google-bert-base-uncased/special_tokens_map.json',
 'drive/MyDrive/google-bert-base-uncased/vocab.txt',
 'drive/MyDrive/google-bert-base-uncased/added_tokens.json')

In [ ]:
trainer_bart.train()

Step,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
200,0.122600,0.768254,0.815469,40.399800,77.129000
400,0.122500,0.768254,0.815469,41.745500,74.643000
600,0.120700,0.768254,0.815469,41.980200,74.225000
800,0.126300,0.768254,0.815469,42.172100,73.888000
1000,0.126700,0.768254,0.815469,42.301100,73.662000
1200,0.110100,0.768254,0.815469,42.099600,74.015000
1400,0.119500,0.768254,0.815469,41.868700,74.423000
1600,0.127800,0.768254,0.815469,41.771200,74.597000
1800,0.132800,0.768254,0.815469,42.473500,73.363000
2000,0.118000,0.768254,0.815469,42.491600,73.332000


TrainOutput(global_step=2037, training_loss=0.1222490745838448, metrics={'train_runtime': 2523.9044, 'train_samples_per_second': 0.807, 'total_flos': 7122235291148196.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 49606, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 1907143, 'train_mem_gpu_alloc_delta': 442944512, 'train_mem_cpu_peaked_delta': 94804109, 'train_mem_gpu_peaked_delta': 6297678848})

In [ ]:
model_path = "drive/MyDrive/mbart-bert-base-uncased"
model2.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('drive/MyDrive/mbart-bert-base-uncased/tokenizer_config.json',
 'drive/MyDrive/mbart-bert-base-uncased/special_tokens_map.json',
 'drive/MyDrive/mbart-bert-base-uncased/vocab.txt',
 'drive/MyDrive/mbart-bert-base-uncased/added_tokens.json')

In [ ]:
def get_prediction(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return probs.argmax().item()

In [ ]:
X_google_test_result = [get_prediction(item) for item in X_google_test['google'].to_list()]


In [ ]:
print(classification_report(Y_google_test, X_google_test_result))

              precision    recall  f1-score   support

         0.0       0.90      0.90      0.90      1786
         1.0       0.38      0.37      0.37       292

    accuracy                           0.83      2078
   macro avg       0.64      0.63      0.64      2078
weighted avg       0.82      0.83      0.83      2078



In [ ]:
def get_prediction_mbart(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model2(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return probs.argmax().item()

In [ ]:
X_mbart_test

,mbart
3722,Every time you tweeted for the Pantheon of ang...
1875,"I was talking about politics, and somebody I k..."
4419,"There's been severe flooding and rainfall, and..."
3066,"The world, don't cry to Hussein, it's not wort..."
1641,Ali Zardari is a well-known gang member who ki...
...,...
1258,"I mean, it's so bad that the Iranian row is so..."
3101,"In this interesting situation, the common deno..."
1262,A wise person can't be cut through a hole twic...
2069,What is it that makes you jump out of bed and ...


In [ ]:
X_mbart_test_result = [get_prediction_mbart(item) for item in X_mbart_test['mbart'].to_list()]

In [ ]:
print(classification_report(Y_mbart_test, X_mbart_test_result))

              precision    recall  f1-score   support

         0.0       0.88      0.94      0.91      1786
         1.0       0.36      0.22      0.27       292

    accuracy                           0.84      2078
   macro avg       0.62      0.58      0.59      2078
weighted avg       0.81      0.84      0.82      2078



In [ ]:
model_without_english = BertForSequenceClassification.from_pretrained(model_name, num_labels=2).to("cuda")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
training_args_google_without_english = TrainingArguments(
    output_dir='./results_google_without_english',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./results_google_without_english',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=200,               # log & save weights each logging_steps
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)

In [ ]:
trainer_google_without_english = Trainer(
    model=model_without_english,                         # the instantiated Transformers model to be trained
    args=training_args_google_without_english,                  # training arguments, defined above
    train_dataset=google_dataset_valid,         # training dataset
    eval_dataset=google_dataset_valid,
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

In [ ]:
!rm -rf ./results_bart

In [ ]:
trainer_google_without_english.train()

Step,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
200,0.261200,0.250724,0.929076,44.423400,70.143000
400,0.279100,0.139554,0.957959,45.742700,68.120000


TrainOutput(global_step=585, training_loss=0.24613858084393364, metrics={'train_runtime': 477.6805, 'train_samples_per_second': 1.225, 'total_flos': 1344819024761616.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 302457, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 598788, 'init_mem_gpu_peaked_delta': 1329002496, 'train_mem_cpu_alloc_delta': 970382, 'train_mem_gpu_alloc_delta': 1356288512, 'train_mem_cpu_peaked_delta': 94077016, 'train_mem_gpu_peaked_delta': 3242928128})

In [ ]:
trainer_google_without_english.evaluate()

{'epoch': 3.0,
 'eval_accuracy': 0.95795892169448,
 'eval_loss': 0.13955430686473846,
 'eval_mem_cpu_alloc_delta': 115315,
 'eval_mem_cpu_peaked_delta': 224651,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 919243264,
 'eval_runtime': 40.7461,
 'eval_samples_per_second': 76.474}

In [ ]:
def get_prediction_google_without_english(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model_without_english(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return probs.argmax().item()

In [ ]:
X_google_test_result_without_engish = [get_prediction_google_without_english(item) for item in X_google_test['google'].to_list()]

In [ ]:
print(classification_report(Y_google_test, X_google_test_result_without_engish))

              precision    recall  f1-score   support

         0.0       0.93      0.84      0.88      1786
         1.0       0.39      0.63      0.49       292

    accuracy                           0.81      2078
   macro avg       0.66      0.74      0.69      2078
weighted avg       0.86      0.81      0.83      2078



In [ ]:
bart_without_english = BertForSequenceClassification.from_pretrained(model_name, num_labels=2).to("cuda")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
training_args_bart_without_english = TrainingArguments(
    output_dir='./results_bart_without_english',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./results_bart_without_english',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=200,               # log & save weights each logging_steps
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)

In [ ]:
trainer_bart_without_english = Trainer(
    model=bart_without_english,                         # the instantiated Transformers model to be trained
    args=training_args_bart_without_english,                  # training arguments, defined above
    train_dataset=bart_dataset_valid,         # training dataset
    eval_dataset=bart_dataset_valid,
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

In [ ]:
trainer_bart_without_english.train()

Step,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
200,0.460100,0.349075,0.855905,42.334900,73.604000
400,0.322800,0.157976,0.948652,42.221200,73.802000


TrainOutput(global_step=585, training_loss=0.33418382823976694, metrics={'train_runtime': 460.0286, 'train_samples_per_second': 1.272, 'total_flos': 1228145228092800.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 54913, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 968379, 'train_mem_gpu_alloc_delta': 1767702528, 'train_mem_cpu_peaked_delta': 94080696, 'train_mem_gpu_peaked_delta': 2814068224})

In [ ]:
trainer_bart_without_english.evaluate()

{'epoch': 3.0,
 'eval_accuracy': 0.9486521181001284,
 'eval_loss': 0.15797649323940277,
 'eval_mem_cpu_alloc_delta': 117884,
 'eval_mem_cpu_peaked_delta': 228691,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 164764160,
 'eval_runtime': 42.3901,
 'eval_samples_per_second': 73.508}

In [ ]:
def get_prediction_bart_without_english(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = bart_without_english(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return probs.argmax().item()

In [ ]:
X_bart_test_result_without_engish = [get_prediction_bart_without_english(item) for item in X_mbart_test['mbart'].to_list()]

In [ ]:
print(classification_report(Y_mbart_test, X_bart_test_result_without_engish))

              precision    recall  f1-score   support

         0.0       0.91      0.89      0.90      1786
         1.0       0.42      0.47      0.44       292

    accuracy                           0.83      2078
   macro avg       0.66      0.68      0.67      2078
weighted avg       0.84      0.83      0.84      2078



In [ ]:
model_path = "drive/MyDrive/google_without_english-bert-base-uncased"
model_without_english.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('drive/MyDrive/google_without_english-bert-base-uncased/tokenizer_config.json',
 'drive/MyDrive/google_without_english-bert-base-uncased/special_tokens_map.json',
 'drive/MyDrive/google_without_english-bert-base-uncased/vocab.txt',
 'drive/MyDrive/google_without_english-bert-base-uncased/added_tokens.json')

In [ ]:
model_path = "drive/MyDrive/bart_without_english-bert-base-uncased"
bart_without_english.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('drive/MyDrive/bart_without_english-bert-base-uncased/tokenizer_config.json',
 'drive/MyDrive/bart_without_english-bert-base-uncased/special_tokens_map.json',
 'drive/MyDrive/bart_without_english-bert-base-uncased/vocab.txt',
 'drive/MyDrive/bart_without_english-bert-base-uncased/added_tokens.json')